In [144]:
# !pip install langchain

In [145]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

In [146]:
client = OpenAI(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"]
)
model_name = "deepseek-ai/DeepSeek-V3"

In [147]:
def get_completion(prompt, model="deepseek-ai/DeepSeek-V3") -> str:
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

In [148]:
message = get_completion("What is 1+1?")
print(message)

The sum of 1 and 1 is **2**.


In [149]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [150]:
style = """American English \
in a calm and respectful tone
"""

In [151]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}, just return the translate result.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
, just return the translate result.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [152]:
response = get_completion(prompt)

In [153]:
print(response)

I’m really frustrated that my blender lid came off and splattered smoothie all over my kitchen walls. To make things even more challenging, the warranty doesn’t cover the cost of cleaning up the mess. I’d really appreciate your assistance with this right away. Thank you!


## Chat API : LangChain

Let's try how we can do the same using LangChain.

### Prompt template

In [154]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [155]:
chat = ChatOpenAI(
    model=model_name,  # 根据实际情况填写模型名称
    openai_api_base=os.environ["BASE_URL"],  # DeepSeek API地址
    temperature=0
)

In [156]:
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000000000CAE4490>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000000000CCBA9E0>, model_name='deepseek-ai/DeepSeek-V3', temperature=0.0, model_kwargs={}, openai_api_key='sk-bsknxjxigbznbrxdxtcagcfrooifxinkmewpkuoacckvexya', openai_api_base='https://api.siliconflow.cn/v1', openai_proxy='')

In [157]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [158]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [159]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [160]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [161]:
customer_style = """American English \
in a calm and respectful tone
"""

In [162]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [163]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [164]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [165]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [166]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [167]:
print(customer_response.content)

Certainly! Here's the text translated into American English with a calm and respectful tone:

"I’m really frustrated that my blender lid came off and splattered smoothie all over my kitchen walls. To make things even more challenging, the warranty doesn’t cover the cost of cleaning up the mess. I’d really appreciate your assistance with this as soon as possible."


In [168]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [169]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [170]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [171]:
service_response = chat.invoke(service_messages)
print(service_response.content)

Ahoy there, matey! I be reachin' out to ye with a bit o' news regardin' yer warranty. Alas, the warranty be not coverin' the costs o' cleanin' yer galley, as 'twas yer own hand that led to the mishap. Ye forgot to secure the lid afore ye set yer blender a-spinnin', and that be on ye. Aye, 'tis a bit o' bad luck, but such be the way o' the seas. Fair winds to ye, and may yer next voyage be smoother! Farewell, me hearty! 🏴‍☠️


## Output Parsers
Let's start with defining how we would like the LLM output to look like:

In [172]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [173]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [174]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [175]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(
    model=model_name,  # 根据实际情况填写模型名称
    openai_api_base=os.environ["BASE_URL"],  # DeepSeek API地址
    temperature=0
)
response = chat(messages)
print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [176]:
type(response.content)

str

### Parse the LLM output string into a Python dictionary

In [177]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [178]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [179]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [180]:
format_instructions = output_parser.get_format_instructions()

In [181]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [182]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown, it should be a boolean, not a string.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1, It should be an integer value, not a string.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,  format_instructions=format_instructions)

In [183]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown, it should be a boolean, not a string.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1, It should be an integer value, not a string.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet forma

In [184]:
response = chat.invoke(messages)

In [185]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [186]:
output_dict = output_parser.parse(response.content)

In [187]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [188]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [189]:
print(output_dict.get('delivery_days'))

2
